In [2]:
import ru_core_news_lg
import spacy as sp
from spacy.symbols import ORTH, LEMMA
from IPython.display import Image
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab
from spacy import displacy

#from spacy.lang.ru import Russian
#nlp = Russian()

nlp = sp.load('ru_core_news_lg')

Объекты-контейнеры библиотеки spaCy

In [3]:
# Получение индекса токена в объекте Doc
doc = nlp("Я хочу миндальную конфету")
[doc[i] for i in range(len(doc))]

# Здесь вызываем конструктор класса Doc и передаем ему два параметра:
# 1) объект vocab()— контейнер хранилища со словарными данными, например типами лексем (прилагательное, глагол, существительное и т. д.)
# 2) и список токенов (words) для добавления в создаваемый объект Doc 
doc = Doc(Vocab(), words=[u'Привет', u'всем'])
doc

Привет всем 

Обход в цикле синтаксических дочерних элементов токена

In [4]:
# Для получения программным образом левосторонних дочерних элементов токена "конфета"
doc = nlp("Я хочу эту миндальную конфету")
print([w for w in doc[3].lefts])

doc = nlp("Я хочу миндальную конфету")
print([w for w in doc[3].lefts])

# Вывод -- работет странно?

# Token.rights  -- > для получения право- сторонних синтаксических дочерних элементов
# Token.children -- > для получения всех дочерних элементов
[w for w in doc[3].children]


[эту]
[миндальную]


[миндальную]

Контейнер doc.sents

In [5]:
# С помощью свойства doc.sents объекта Doc текст можно разделить на отдельные предложения
doc = nlp(u'В небе запахло грозой. Скоро будет дождь. Пойдем книжку читать!')
for sent in doc.sents:
    [sent[i] for i in range(len(sent))]
    print(sent)

# вспомним метод .pos_
for token in doc:
    print('%12s %12s %12s' % (token.text, token.pos_, token.dep_))


# количество предложений оканчивающихся галаголом
counter = 0
for sent in doc.sents:
    if sent[len(sent)-2].pos_ == 'VERB':
        counter+=1
print(counter)

В небе запахло грозой.
Скоро будет дождь.
Пойдем книжку читать!
           В          ADP         case
        небе         NOUN          obl
     запахло         VERB         ROOT
      грозой         NOUN          obl
           .        PUNCT        punct
       Скоро          ADV       advmod
       будет         VERB         ROOT
       дождь         NOUN        nsubj
           .        PUNCT        punct
      Пойдем         VERB         ROOT
      книжку         NOUN          obj
      читать         VERB        xcomp
           !        PUNCT        punct
1


#### Контейнер doc.noun_chunks
Именной фрагмент (noun chunk) — это фраза, главным элементом которой является существительное

In [6]:
doc = nlp(u'В небе запахло грозой. Скоро будет дождь. Пойдем книжку читать! Мир чудесен.')

for token in doc:
    if token.pos_=='NOUN':
        chunk = ''
        for w in token.children:
            if w.pos_ == 'DET' or w.pos_ == 'ADJ':
                chunk = chunk + w.text + ' '
            chunk = chunk + token.text
            print(chunk)

print("-" * 40)
for token in doc:
    if token.pos_=='NOUN':
        chunk = ''
        for w in token.lefts:
            chunk = chunk + token.text
            print(chunk)

небе
----------------------------------------
небе


#### Объект Span
Объект Span (от англ. span — «интервал») представляет собой часть объекта Doc

In [7]:
doc = nlp(u'В небе запахло грозой.')
#print(doc[2:])

# Span включает несколько методов, самый интересный из которых — span.retokenize().
# С его помощью интервал можно объединять в единый токен, производя повторную токенизацию документа. 
# Это удобно, когда текст содержит названия из нескольких слов.

doc = nlp(u'Мост Золотые Ворота - культовая достопримечательность Сан-Франциско')
for token in doc:
    print('%12s %12s %12s %12s' % (token.text, token.pos_, token.dep_,token.head.text))

span = doc[0:3]
span2 = doc[6:9]

with doc.retokenize() as retokenizer:
    retokenizer.merge(span)
    retokenizer.merge(span2)

print("-" * 40)
for token in doc:
    print('%12s %12s %12s %12s' % (token.text, token.pos_, token.dep_,token.head.text))


        Мост         NOUN         ROOT         Мост
     Золотые          ADJ         amod         Мост
      Ворота        PROPN        appos         Мост
           -        PUNCT         amod достопримечательность
   культовая          ADJ         amod достопримечательность
достопримечательность         NOUN        appos         Мост
         Сан        PROPN         nmod достопримечательность
           -        PROPN         nmod достопримечательность
   Франциско        PROPN         nmod достопримечательность
----------------------------------------
Мост Золотые Ворота         NOUN         ROOT Мост Золотые Ворота
           -        PUNCT         amod достопримечательность
   культовая          ADJ         amod достопримечательность
достопримечательность         NOUN        appos Мост Золотые Ворота
Сан-Франциско        PROPN         nmod достопримечательность


#### Настройка конвейера обработки текста под свои нужды

In [8]:
from spacy import util

print(nlp.pipe_names)

# Отключение компонентов конвейера
nlp = sp.load('ru_core_news_lg', disable=['parser'])

# В данном случае мы создадим конвейер обработки без утилиты разбора зависимостей. 
doc = nlp(u'Я хочу кислое зеленое яблоко')
print("-" * 40)
for token in doc:
    print('%12s %12s %12s' % (token.text, token.pos_, token.dep_))

# метки зависимостей выведены не были.

# полное название использованной модели
print("Название использованной модели:", nlp.meta['lang'] + '_' + nlp.meta['name'])
print("Путь:", util.get_package_path('ru_core_news_lg'))
nlp = sp.load('ru_core_news_lg')

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
----------------------------------------
           Я         PRON             
        хочу         VERB             
      кислое          ADJ             
     зеленое          ADJ             
      яблоко         NOUN             
Название использованной модели: ru_core_news_lg
Путь: /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ru_core_news_lg


In [9]:
# какие компоненты поддерживаются в контексте данной модели и могут быть загружены в конвейер
print(nlp.meta['pipeline'])

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


Настройка компонентов конвейера под свои нужды

In [10]:
import random

doc = nlp(u'Замечательный день сходить в Яблоко за кольцом')
for ent in doc.ents:
    print(ent.text, ent.label_)
# Метка ORG обозначает различные компании, государственные бюро и прочие учреждения. 
# Но нам нужно, чтобы средство распознавания сущностей классифицировало его как сущность типа КОМПАНИЯ.

LABEL = 'COMPANY'

# правая граница не входит !!!
TRAIN_DATA = [
    ('Замечательный день сходить в Яблоко за кольцом', {'entities': [(29, 34, 'COMPANY')]}),
    ('Приготовила сегодня пирог с яблоком', {'entities': []}),
    ('Потратила все деньги в Яблоке.', {'entities': [(23, 29, 'КОМПАНИЯ')]}),
]

ner = nlp.get_pipe('ner')
ner.add_label(LABEL)

Яблоко LOC


1

In [11]:
meta_on = nlp.meta['pipeline'] #список компонентов конвейера, используемых с моделью
print(meta_on)

# отключаем компоненты 
for component in meta_on:
    if component != 'ner':
        nlp.disable_pipes(component)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [12]:
from spacy.training.example import Example

for i in range(25):
    random.shuffle(TRAIN_DATA)
    for batch in sp.util.minibatch(TRAIN_DATA, size = 3):
        for text, annotations in batch:
            # create Example
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Update the model
            nlp.update([example])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Замечательный день сходить в Яблоко за кольцом" with entities "[(29, 34, 'COMPANY')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [15]:
doc = nlp(u'Потратила все деньги в Яблоке')
for ent in doc.ents:
    print(ent.text, ent.label_)

doc = nlp(u'Съела все Яблоки')
for ent in doc.ents:
    print(ent.text, ent.label_)

Яблоке КОМПАНИЯ
Яблоки КОМПАНИЯ


Учтите: внесенные только что обновления будут утрачены сразу после закрытия текущего сеанса интерпретатора Python. Для решения этой проблемы в классе Pipe, родительском для класса EntityRecognizer и других классов компонентов конвейера, предусмотрен метод to_ disk(), предназначенный для сериализации конвейера на диск:

In [17]:
''' 
ner.to_disk('/usr/to/ner')

import spacy
from spacy.pipeline import EntityRecognizer
nlp = spacy.load('en', disable=['ner'])

ner = EntityRecognizer(nlp.vocab)
ner.from_disk('/usr/to/ner')
nlp.add_pipe(ner)

'''

" \nner.to_disk('/usr/to/ner')\n\nimport spacy\nfrom spacy.pipeline import EntityRecognizer\nnlp = spacy.load('en', disable=['ner'])\n\nner = EntityRecognizer(nlp.vocab)\nner.from_disk('/usr/to/ner')\nnlp.add_pipe(ner)\n\n"

Вы загрузили модель, отключив ее компонент ner по умолчанию. Затем создали новый экземпляр ner, после чего загрузили данные с диска. Добавили компонент ner в конвейер обработки